In [5]:
# train, test on MEBeauty and plot the results

import mebeauty_dataset
import argparse
from progiter import ProgIter
import torch
import time
from torchvision import models
import matplotlib.pyplot as plt
from torch import nn
import torch.optim as optim
from __future__ import print_function, division

import torch.nn as nn
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode



def train_model(model, traindata, testdata, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):

        my_loss_final = 0
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            l = 0
            if phase == 'train':
                dataloader = traindata
                dataset_sizes = len(traindata)
                model.train()  # Set model to training mode
            else:
                dataloader = testdata
                dataset_sizes = len(testdata)
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                k=0
                my_loss = 0
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    print(outputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.float())
                    #print(outputs)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                #my_loss_final += my_loss/k
                #print(loss.item(),inputs.size(0),dataset_sizes)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            #epoch_loss = running_loss / dataset_sizes
            #epoch_loss = my_loss_final / 1
            epoch_acc = running_corrects.double() / dataset_sizes

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


g=1
if g == 1:  
    
    base_model = "vgg16"
    train_aug = False
    train_scores = 'scores/train_universal_scores.csv'
    test_scores = 'scores/test_universal_scores.csv'
    batch = 8
    epochs = 10
   
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    traindata, testdata = mebeauty_dataset.train_test_data(train_scores, test_scores, train_augmentation = train_aug, 
                                                           batch = batch) # train and test dataloaders
    
    #model, criterion, optimizer = model_preparation(base_model, device)
    
    #train_loss, val_loss = fit(model, criterion, optimizer, traindata, testdata, device, epochs)
    
    
    

    
    model_ft = models.vgg16(pretrained = True, progress = False)
    in_features  = 25088
        
    
    #model = models.vgg16(pretrained = True, progress = False)
    model_ft.classifier = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=1, bias=True)
    )

    for param in model_ft.features.parameters():
        param.requires_grad = False

    model_ft = model_ft.to(device)

    #criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
    optimizer = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
    #exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    
    
    
    
    criterion = nn.MSELoss()
    #optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

    #exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    
    xxx = train_model(model_ft, traindata, testdata, criterion, optimizer,
                       num_epochs=10)
    #best_acc = train_model(model, traindata, testdata, criterion, optimizer, num_epochs=10, is_inception=False)
     #mebeauty_dataset.plot_training(train_loss, val_loss)

Epoch 0/9
----------


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 7.44 GiB total capacity; 5.85 GiB already allocated; 88.31 MiB free; 5.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

    import torch
    torch.cuda.empty_cache()    

In [ ]:
    import gc
    del variables
    gc.collect()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)